In [1]:
import pandas as pd
import numpy as np

In [2]:
#load dataset 
#SPMF datasets contain 1 sequance per line. Every sequance (and line) ands with -2. Every itemset ends with -1
file_path = "dataSets/BIBLE.txt"

sequences = []
with open(file_path, 'r') as file:
    for line in file:
        line = line.replace(" -2", "")
        itemSets = []
        for set in line.strip().split("-1"):
            itemSet = []
            for item in set.split():
                itemSet.append(int(item))
            itemSets.append(itemSet)
        sequences.append(itemSets)

KeyboardInterrupt: 